In [ ]:
from pathlib import Path
from Bio.PDB import PDBParser, PDBIO, NeighborSearch
from Bio.PDB import Selection
from utils.docking import sdf2centroid

In [ ]:
def extract_res(pdb_file, center:list, threshold=10.0):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('retract', pdb_file)

    atoms = Selection.unfold_entities(structure, 'A')
    ns = NeighborSearch(atoms)

    nearby_residues = set()
    
    for atom in ns.search(center, threshold):
        nearby_residues.add(atom.get_parent())

    # Create a new structure with only the nearby residues
    new_structure = structure.copy()
    for model in new_structure:
        for chain in model:
            for residue in list(chain):
                if residue not in nearby_residues:
                    chain.detach_child(residue.id)

    output_file = pdb_file.replace('.pdb', '_pocket10.pdb')
    io = PDBIO()
    io.set_structure(new_structure)
    io.save(str(output_file))
    print(f"Saved {output_file}")

In [ ]:
center = sdf2centroid('../../Targets/BRD4/BRD4_8pxa_ligand_A.sdf')
pdb_file = '../../Targets/BRD4/BRD4_8pxa_rec_A.pdb'
extract_res(pdb_file, center)